## Register an External Model

1. Register the Subro model as a multiclass classifier with target values `low, medium, high`
2. Create the deployment
4. Update deployment settings: prediction tracking, accuracy tracking (optional)

This is a one and done exercise

We'll write the deployment ID to disk so we can grab it later for monitoring purposes, see `03. Score and Monitor.ipynb`

In [0]:

import datarobot as dr
import yaml
ext_reg_model = dr.RegisteredModelVersion.create_for_external(
        name = "Subro model", 
        target = {"type": "Multiclass", "name": "subro_predictions", "classNames" : ["low", "medium", "high"]},
        # datasets = {"trainingDataCatalogId": "67bf7d05ad1d7cda48cc2453"}, 
        registered_model_name = "Subro model Codespace hosted",
        registered_model_description="Subro model hosted in DR Codespace"
    )
deployment = dr.Deployment.create_from_registered_model_version(
        model_package_id = ext_reg_model.id,
        label=ext_reg_model.name,
        description=f"external model deployment for {ext_reg_model.name}",
        prediction_environment_id='65f08b280c919297b297039c'
    )
# enable prediction tracking
deployment.update_drift_tracking_settings(target_drift_enabled=True, feature_drift_enabled=False)
# Enabling Accuracy
deployment.update_association_id_settings(column_names=["ASSOCIATION_ID"], required_in_prediction_requests=False)
# Enabling Challenger
deployment.update_predictions_data_collection_settings(enabled=True)

with open("deployment.yaml", "w") as f:
    f.write(yaml.dumps( dict(deployment_id = deployment.id))